In [1]:
from google.colab import drive
drive.mount('/content/drive')
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({'id':'174Ws0LnaEQdK7UmOKh-8GJwftO18Mfej'})
your_module.GetContentFile('metric.py')
your_module1 = drive.CreateFile({'id':'11ssSCDodnZ6_jXcHLZ8AqYHbg7g7K8Be'})
your_module1.GetContentFile('threshold.py')
your_module2 = drive.CreateFile({'id':'1JpuTOGEKMq9iZcO6Tp4ET9icd5EG0P0D'})
your_module2.GetContentFile('cat_list.py')
your_module2 = drive.CreateFile({'id':'1gsMrGL0BfVASQICf-eesMRelmCgMLTYs'})
your_module2.GetContentFile('cat_list_show.py')
from metric import *
from threshold import *
from cat_list import *
from cat_list_show import *

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
from keras import backend as K
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, LSTM,Conv1D
from tensorflow.keras.optimizers import Adam,Adagrad,Nadam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout,GlobalMaxPooling1D,Conv1D,BatchNormalization
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras import initializers, regularizers, constraints
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score, roc_curve,auc
from sklearn.metrics import precision_score
def accuracy(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())
def fbeta(y_true, y_pred, threshold_shift=0.005):
    beta = 2
    y_pred = K.clip(y_pred, 0, 1)
    y_pred_bin = K.round(y_pred + threshold_shift)
    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))

def ffb (y_test, predictions, beta=1):
  pr = precision(y_test, predictions)
  re = recall(y_test, predictions)
  num = float((1+pow(beta,2))*pr*re)
  den = float(pow(beta,2)*pr + re)
  ffb = num/den
  return ffb
def accuracy3(y_test, predictions):
    accuracy = 0.0

    for i in range(y_test.shape[0]):
        intersection = 0.0
        union = 0.0
        for j in range(y_test.shape[1]):
            if int(y_test[i,j]) == 1 or int(predictions[i,j]) == 1:
                union += 1
            if int(y_test[i,j]) == 1 and int(predictions[i,j]) == 1:
                intersection += 1

            
        if union != 0:
            accuracy = accuracy + float(intersection/union)

    accuracy = float(accuracy/y_test.shape[0])

    return accuracy
#فراخوانی داده های آموزشی و آزمایشی
dataset = pd.read_excel('drive/My Drive/2020/c1.xlsx')
test_file = pd.read_excel('drive/My Drive/2020/test_cat_plo.xlsx')
S_comments_labels = dataset[['بازیگر#مثبت','بازیگر#منفی','بازیگری#مثبت','بازیگری#منفی','داستان#مثبت',
                              'داستان#منفی','دیالوگ#مثبت','دیالوگ#منفی','سبک#مثبت','سبک#منفی','فیلم#مثبت','فیلم#منفی',
                             'فیلم_نامه#مثبت','فیلم_نامه#منفی','فیلمبرداری#مثبت','فیلمبرداری#منفی',
                             'لوکیشن#مثبت','لوکیشن#منفی','محتوا#مثبت','محتوا#منفی','موسیقی#مثبت',
                             'موسیقی#منفی','موضوع#مثبت','موضوع#منفی','کارگردان#مثبت',
                             'کارگردان#منفی','گریم#مثبت','گریم#منفی']]

S_test_comments_labels = test_file[['بازیگر#مثبت','بازیگر#منفی','بازیگری#مثبت','بازیگری#منفی','داستان#مثبت',
                              'داستان#منفی','دیالوگ#مثبت','دیالوگ#منفی','سبک#مثبت','سبک#منفی','فیلم#مثبت','فیلم#منفی',
                             'فیلم_نامه#مثبت','فیلم_نامه#منفی','فیلمبرداری#مثبت','فیلمبرداری#منفی',
                             'لوکیشن#مثبت','لوکیشن#منفی','محتوا#مثبت','محتوا#منفی','موسیقی#مثبت',
                             'موسیقی#منفی','موضوع#مثبت','موضوع#منفی','کارگردان#مثبت',
                             'کارگردان#منفی','گریم#مثبت','گریم#منفی']]
category_list=['بازیگر#مثبت','بازیگر#منفی','بازیگری#مثبت','بازیگری#منفی','داستان#مثبت',
                              'داستان#منفی','دیالوگ#مثبت','دیالوگ#منفی','سبک#مثبت','سبک#منفی','فیلم#مثبت','فیلم#منفی',
                             'فیلم_نامه#مثبت','فیلم_نامه#منفی','فیلمبرداری#مثبت','فیلمبرداری#منفی',
                             'لوکیشن#مثبت','لوکیشن#منفی','محتوا#مثبت','محتوا#منفی','موسیقی#مثبت',
                             'موسیقی#منفی','موضوع#مثبت','موضوع#منفی','کارگردان#مثبت',
                             'کارگردان#منفی','گریم#مثبت','گریم#منفی']
stop_words =['به','شکل','ممکن','میشه','دلیل','رو','فقط','در','خانم','آقای','اگر','مریلا',
            'این','هم','اسامی','زارعی','بهرام','رادان','حیایی','پارسا','پیروزفر','و','صابر','ابر','حال','های',
            'ها','یه','ای','بابت','آبیار','از','تا','هفتاد','درصد','الناز','دو','اول','شاکردوست','سیمرغ','زن','تاریخ','شاکر','هاش','ریگی','اش','معادی','نرگس','دوم','ابیار','پیمان','اقای','بهداد','مهناز','حامد','عطاران','علی','نوید','ام','امیر','۱۰','محمدزاده','نفر','تیکت','سعید','لباس','خدا','هانیه'
             ,'توسلی','سحر','۲۰','ثقفی','رضا','کریمی','روسی','۴','کامنت','عبدالمالک','ش','اکبر','هیس','مهرجویی','طباطبایی','سرخپوست','عبدی','شش','شیطان','پرستی'
            ]
#پیش پردازش
import re
def preprocess_text(sentence):
    # Removing multiple spaces
    sentence = re.sub(r'@\S+', '', sentence)
    sentence = re.sub(r'!\S+', '', sentence)
    sentence = re.sub(r'؟\S+', '', sentence)
    sentence = re.sub(r'[.]', ' ', sentence)
    sentence = re.sub(r'[/]', '', sentence)
    sentence = re.sub(r'[،]', ' ', sentence)
    sentence = re.sub(r'[؛]', '', sentence)
    sentence = sentence.split()
    sentence =[word for word in sentence if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence
def ReplacetwoMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1", s)
dataset['clean_text'] = dataset['Sentence'].apply(ReplacetwoMore)
test_file['clean_text'] = test_file['Sentence'].apply(ReplacetwoMore)
X = []
sentences = list(dataset["clean_text"])
for sen in sentences:
    X.append(preprocess_text(sen))    
X_t = []
sentences2 = list(test_file["clean_text"])
for sen in sentences2:
    X_t.append(preprocess_text(sen))
y = S_comments_labels.values
y_test1=S_test_comments_labels.values
target =y
data = X
target1 = y_test1
data1 = X_t
x_train, x_test = data, data1
y_train, y_test = target, target1
#شمارش تعداد کلمات منحصر به فرد
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
#شمارش بزرگترین طول متن
r_len=[]
for text in data:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)   
MAX_REVIEW_LEN=np.max(r_len)
num_words = num_unique_word
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
max_tokens = MAX_REVIEW_LEN
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,padding='post')
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
x1=[]
x2=[]
x3=[]
x4=[]
x5=[]
x6=[]
x7=[]
x8=[]
x9=[]
x10=[]
x11=[]
x12=[]
x13=[]
x14=[]
x15=[]
x16=[]
x17=[]
x18=[]
x19=[]
x20=[]
x21=[]
x22=[]
x23=[]
x24=[]
x25=[]
x26=[]
x27=[]
x28=[]
x29=[]
x30=[]
x31=[]
x32=[]
x33=[]
x34=[]
x35=[]
x36=[]
x37=[]
x38=[]
x39=[]
x40=[]
i=1
import time
start_time = time.time()
for i in range(50):
  embedding_size = 300
  model4 = tf.keras.Sequential()
  model4.add(Embedding(input_dim=num_words,
                      output_dim=embedding_size,
                      input_length=max_tokens,
                      name='embedding_layer'))
  model4.add(Dropout(0.5))
  model4.add(LSTM(units=256, return_sequences=True))
  model4.add(BatchNormalization())
  model4.add(GlobalMaxPooling1D())
  model4.add(Dense(200, activation='relu'))
  model4.add(Dropout(0.5))
  model4.add(Dense(28,activation='sigmoid'))
  optimizer =Nadam(learning_rate=1e-3)
  model4.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=[accuracy,fbeta])
  history=model4.fit(x_train_pad, y_train, epochs=20, batch_size=50,shuffle=False,verbose=0)
  p = model4.predict(x_test_pad)
  th(p,0.5)
  prr = model4.predict(x_test_pad)
  prr[prr>=0.5]=1
  prr[prr<0.5]=0
  ac=subsetAccuracy(y_test, p)
  hm=hammingLoss(y_test, p)
  acc=accuracy3(y_test, p)
  pm=precisionMacro(y_test, p)
  pr=precisionMicro(y_test, p)
  rm=recallMacro(y_test, p)
  rmm=recallMicro(y_test, p)
  fbm1=fbetaMacro(y_test, p, 1)
  fbmm1=fbetaMicro(y_test, p, 1)
  pmq=precision_score(y_test, prr, average='macro')
  prq=precision_score(y_test, prr, average='macro')
  fb=ffb(y_test, p, 1)
  fbq=ffb(y_test, prr, 1)
  aucc1=roc_auc_score(y_test, p,average='micro')
  aucc2=roc_auc_score(y_test, p,average='macro')     
  acq=subsetAccuracy(y_test, prr)
  hmq=hammingLoss(y_test, prr)
  accq=accuracy3(y_test, prr)



  am=accuracyMacro(y_test, p)
  amm=accuracyMicro(y_test, p)
  amq=accuracyMacro(y_test, prr)
  ammq=accuracyMicro(y_test, prr)
  pmq=precision_score(y_test, prr, average='macro')
  prq=precision_score(y_test, prr, average='micro')

  rmq=recall_score(y_test, prr, average='macro')
  rmmq=recall_score(y_test, prr, average='micro')
  fbm=fbeta_score(y_test, prr, average='macro', beta=1)
  fbmm=fbeta_score(y_test, prr, average='micro', beta=1)
  aucc11=roc_auc_score(y_test, prr,average='micro')
  aucc22=roc_auc_score(y_test, prr,average='macro')
  ru=recall(y_test, p)
  ruj=recall(y_test, prr)
  py=precision(y_test, p)
  pyq=precision(y_test, prr)

  okl=oneError(y_test, p)
  rankl=rankingLoss(y_test, p)
  cokl=coverage(y_test, p)
  okl1=oneError(y_test, prr)
  rankl1=rankingLoss(y_test, prr)
  cokl1=coverage(y_test, prr)













  print("%s: %.5f%%" % ('subset Accuracy tpc', ac))
  x1.append(ac)
  print("%s: %.5f%%" % ('subset Accuracy', acq))
  x2.append(acq)

  print("%s: %.5f%%" % ('hamming Loss tpc', hm))
  x3.append(hm)
  print("%s: %.5f%%" % ('hamming Loss ', hmq))
  x4.append(hmq)

  print("%s: %.5f%%" % ('Accuracy tpc', acc))
  x5.append(acc)
  print("%s: %.5f%%" % ('Accuracy', accq))
  x6.append(accq)
  print("%s: %.5f%%" % ('precision tpc', py))
  x7.append(py)
  print("%s: %.5f%%" % ('precision', pyq))
  x8.append(pyq)

  print("%s: %.5f%%" % ('recall tpc', ru))
  x9.append(ru)
  print("%s: %.5f%%" % ('recall ', ruj))
  x10.append(ruj)

  print("%s: %.5f%%" % ('fbeta tpc', fb))
  x11.append(fb)
  print("%s: %.5f%%" % ('fbeta', fbq))
  x12.append(fbq)



  print("%s: %.5f%%" % ('accuracyMacro  tpc', am))
  x21.append(am)
  print("%s: %.5f%%" % ('accuracyMacro', amq))
  x22.append(amq)

  print("%s: %.5f%%" % ('accuracyMicro tpc', amm))
  x23.append(amm)
  print("%s: %.5f%%" % ('accuracyMicro', ammq))
  x24.append(ammq)





  print("%s: %.5f%%" % ('precisionMacro tpc', pm))
  x25.append(pm)
  print("%s: %.5f%%" % ('precisionMacro ', pmq))
  x26.append(pmq)

  print("%s: %.5f%%" % ('precisionMicro tpc', pr))
  x27.append(pr)
  print("%s: %.5f%%" % ('precisionMicro', prq))
  x28.append(prq)

  print("%s: %.5f%%" % ('recallMacro tpc', rm))
  x29.append(rm)
  print("%s: %.5f%%" % ('recallMacro', rmq))
  x30.append(rmq)

  print("%s: %.5f%%" % ('recallMicro tpc', rmm))
  x31.append(rmm)
  print("%s: %.5f%%" % ('recallMicro', rmmq))
  x32.append(rmmq)


  print("%s: %.5f%%" % ('fbetaMacro  tpc', fbm1))
  x33.append(fbm1)
  print("%s: %.5f%%" % ('fbetaMacro', fbm))
  x34.append(fbm)

  print("%s: %.5f%%" % ('fbetaMicro tpc', fbmm1))
  x35.append(fbmm1)
  print("%s: %.5f%%" % ('fbetaMicro', fbmm))
  x36.append(fbmm)

  print("%s: %.5f%%" % ('roc_auc macro  tpc', aucc2))
  x37.append(aucc2)
  print("%s: %.5f%%" % ('roc_auc macro', aucc22))
  x38.append(aucc22)

  print("%s: %.5f%%" % ('roc_auc micro tpc', aucc1))
  x39.append(aucc1)
  print("%s: %.5f%%" % ('roc_auc micro', aucc11))
  x40.append(aucc11)

  print("%s: %.5f%%" % ('oneError tpc', okl))
  x13.append(okl)
  print("%s: %.5f%%" % ('oneError', okl1))
  x14.append(okl1)

  print("%s: %.5f%%" % ('rankingLoss tpc', rankl))
  x15.append(rankl)
  print("%s: %.5f%%" % ('rankingLoss ', rankl1))
  x16.append(rankl1)

  print("%s: %.5f%%" % ('coverage tpc', cokl))
  x17.append(cokl)
  print("%s: %.5f%%" % ('coverage', cokl1))
  x18.append(cokl1)

  print("**********************************")
  print("****************","epoch--->>",i,"**************")
  print("**********************************")
  print("**********************************")





seconds = time.time() - start_time
print('Time Taken:', time.strftime("%H:%M:%S",time.gmtime(seconds)))
print("**********************************")
print("**********************************")
print('subset Accuracy tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x1)*100, np.std(x1)*100))
print('subset Accuracy',"%.5f%% (+/- %.5f%%)" % (np.mean(x2)*100, np.std(x2)*100))

print('hamming Loss tpc',"%.5f (+/- %.5f)" % (np.mean(x3), np.std(x3)))
print('hamming Loss',"%.5f (+/- %.5f)" % (np.mean(x4), np.std(x4)))

print('Accuracy tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x5)*100, np.std(x5)*100))
print('Accuracy',"%.5f%% (+/- %.5f%%)" % (np.mean(x6)*100, np.std(x6)*100))

print('precision  tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x7)*100, np.std(x7)*100))
print('precision ',"%.5f%% (+/- %.5f%%)" % (np.mean(x8)*100, np.std(x8)*100))

print('recall tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x9)*100, np.std(x9)*100))
print('recall',"%.5f%% (+/- %.5f%%)" % (np.mean(x10)*100, np.std(x10)*100))

print('fbeta tpc',"%.5f (+/- %.5f)" % (np.mean(x11), np.std(x11)))
print('fbeta',"%.5f (+/- %.5f)" % (np.mean(x12), np.std(x12)))


print('accuracyMacro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x21)*100, np.std(x21)*100))
print('accuracyMacro',"%.5f%% (+/- %.5f%%)" % (np.mean(x22)*100, np.std(x22)*100))

print('accuracyMicro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x23)*100, np.std(x23)*100))
print('accuracyMicro',"%.5f%% (+/- %.5f%%)" % (np.mean(x24)*100, np.std(x24)*100))

print('precisionMacro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x25)*100, np.std(x25)*100))
print('precisionMacro',"%.5f%% (+/- %.5f%%)" % (np.mean(x26)*100, np.std(x26)*100))

print('precisionMicro tpc',"%.5f (+/- %.5f)" % (np.mean(x27), np.std(x27)))
print('precisionMicro',"%.5f (+/- %.5f)" % (np.mean(x28), np.std(x28)))

print('recallMacro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x29)*100, np.std(x29)*100))
print('recallMacro',"%.5f%% (+/- %.5f%%)" % (np.mean(x30)*100, np.std(x30)*100))

print('recallMicro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x31)*100, np.std(x31)*100))
print('recallMicro',"%.5f%% (+/- %.5f%%)" % (np.mean(x32)*100, np.std(x32)*100))

print('fbetaMacro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x33)*100, np.std(x33)*100))
print('fbetaMacro',"%.5f%% (+/- %.5f%%)" % (np.mean(x34)*100, np.std(x34)*100))

print('fbetaMicro tpc',"%.5f (+/- %.5f)" % (np.mean(x35), np.std(x35)))
print('fbetaMicro',"%.5f (+/- %.5f)" % (np.mean(x36), np.std(x36)))

print('roc_auc macro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x37)*100, np.std(x37)*100))
print('roc_auc macro',"%.5f%% (+/- %.5f%%)" % (np.mean(x38)*100, np.std(x38)*100))

print('roc_auc micro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x39)*100, np.std(x39)*100))
print('roc_auc micro',"%.5f%% (+/- %.5f%%)" % (np.mean(x40)*100, np.std(x40)*100))
print('oneError tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x13)*100, np.std(x13)*100))
print('oneError',"%.5f%% (+/- %.5f%%)" % (np.mean(x14)*100, np.std(x14)*100))

print('rankingLoss tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x15)*100, np.std(x15)*100))
print('rankingLoss',"%.5f%% (+/- %.5f%%)" % (np.mean(x16)*100, np.std(x16)*100))

print('coverage tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(x17)*100, np.std(x17)*100))
print('coverage',"%.5f%% (+/- %.5f%%)" % (np.mean(x18)*100, np.std(x18)*100))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subset Accuracy tpc: 0.55500%
subset Accuracy: 0.55000%
hamming Loss tpc: 0.02839%
hamming Loss : 0.02821%
Accuracy tpc: 0.67480%
Accuracy: 0.68230%
precision tpc: 0.73792%
precision: 0.74292%
recall tpc: 0.70767%
recall : 0.73017%
fbeta tpc: 0.72248%
fbeta: 0.73649%
accuracyMacro  tpc: 0.97161%
accuracyMacro: 0.97179%
accuracyMicro tpc: 0.97161%
accuracyMicro: 0.97179%
precisionMacro tpc: 0.44464%
precisionMacro : 0.56556%
precisionMicro tpc: 0.66887%
precisionMicro: 0.76667%
recallMacro tpc: 0.58423%
recallMacro: 0.46788%
recallMicro tpc: 0.77395%
recallMicro: 0.68543%
fbetaMacro  tpc: 0.47147%
fbetaMacro: 0.48607%
fbetaMicro tpc: 0.71758%
fbetaMicro: 0.72378%
roc_auc macro  tpc: 0.71521%
roc_auc macro: 0.72646%
roc_auc micro tpc: 0.82887%
roc_auc micro: 0.83677%
oneError tpc: 0.96500%
oneError: 0.96500%
rankingLoss tpc: 0.29553%
rankingLoss : 0.27424%
coverage tpc: 20.31000%
coverage: 20.68500%
**********************************
**************** epoch--->> 0 **************
*********

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subset Accuracy tpc: 0.60000%
subset Accuracy: 0.59500%
hamming Loss tpc: 0.02411%
hamming Loss : 0.02357%
Accuracy tpc: 0.71633%
Accuracy: 0.72583%
precision tpc: 0.77125%
precision: 0.77958%
recall tpc: 0.75392%
recall : 0.77892%
fbeta tpc: 0.76248%
fbeta: 0.77925%
accuracyMacro  tpc: 0.97589%
accuracyMacro: 0.97643%
accuracyMicro tpc: 0.97589%
accuracyMicro: 0.97643%
precisionMacro tpc: 0.52840%
precisionMacro : 0.60813%
precisionMicro tpc: 0.73841%
precisionMicro: 0.79720%
recallMacro tpc: 0.61467%
recallMacro: 0.54062%
recallMicro tpc: 0.79928%
recallMicro: 0.75497%
fbetaMacro  tpc: 0.53398%
fbetaMacro: 0.54202%
fbetaMicro tpc: 0.76764%
fbetaMicro: 0.77551%
roc_auc macro  tpc: 0.75760%
roc_auc macro: 0.76352%
roc_auc micro tpc: 0.86392%
roc_auc micro: 0.87201%
oneError tpc: 0.97500%
oneError: 0.97500%
rankingLoss tpc: 0.24979%
rankingLoss : 0.22601%
coverage tpc: 21.30500%
coverage: 21.49000%
**********************************
**************** epoch--->> 1 **************
*********

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subset Accuracy tpc: 0.57000%
subset Accuracy: 0.55500%
hamming Loss tpc: 0.02643%
hamming Loss : 0.02696%
Accuracy tpc: 0.69813%
Accuracy: 0.69313%
precision tpc: 0.76458%
precision: 0.75958%
recall tpc: 0.73183%
recall : 0.73683%
fbeta tpc: 0.74785%
fbeta: 0.74804%
accuracyMacro  tpc: 0.97357%
accuracyMacro: 0.97304%
accuracyMicro tpc: 0.97357%
accuracyMicro: 0.97304%
precisionMacro tpc: 0.45853%
precisionMacro : 0.54095%
precisionMicro tpc: 0.69868%
precisionMicro: 0.77656%
recallMacro tpc: 0.55895%
recallMacro: 0.46363%
recallMicro tpc: 0.78731%
recallMicro: 0.70199%
fbetaMacro  tpc: 0.49073%
fbetaMacro: 0.48576%
fbetaMicro tpc: 0.74035%
fbetaMicro: 0.73739%
roc_auc macro  tpc: 0.72235%
roc_auc macro: 0.72450%
roc_auc micro tpc: 0.84396%
roc_auc micro: 0.84524%
oneError tpc: 0.97500%
oneError: 0.97500%
rankingLoss tpc: 0.27173%
rankingLoss : 0.26747%
coverage tpc: 21.14500%
coverage: 21.03500%
**********************************
**************** epoch--->> 47 **************
********

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subset Accuracy tpc: 0.56500%
subset Accuracy: 0.56500%
hamming Loss tpc: 0.02607%
hamming Loss : 0.02554%
Accuracy tpc: 0.68717%
Accuracy: 0.69467%
precision tpc: 0.77167%
precision: 0.77917%
recall tpc: 0.71767%
recall : 0.73267%
fbeta tpc: 0.74369%
fbeta: 0.75520%
accuracyMacro  tpc: 0.97393%
accuracyMacro: 0.97446%
accuracyMicro tpc: 0.97393%
accuracyMicro: 0.97446%
precisionMacro tpc: 0.43517%
precisionMacro : 0.61943%
precisionMicro tpc: 0.67219%
precisionMicro: 0.81423%
recallMacro tpc: 0.61943%
recallMacro: 0.44648%
recallMicro tpc: 0.81200%
recallMicro: 0.68212%
fbetaMacro  tpc: 0.48585%
fbetaMacro: 0.49641%
fbetaMicro tpc: 0.73551%
fbetaMicro: 0.74234%
roc_auc macro  tpc: 0.71170%
roc_auc macro: 0.71735%
roc_auc micro tpc: 0.83166%
roc_auc micro: 0.83662%
oneError tpc: 0.96000%
oneError: 0.96000%
rankingLoss tpc: 0.28545%
rankingLoss : 0.27101%
coverage tpc: 21.34500%
coverage: 21.60000%
**********************************
**************** epoch--->> 48 **************
********

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
